Agents are systems that use an LLM as a reasoning engineer to determine which actions to take and what the inputs to those actions should be. The results of those actions can then be fed back into the agent and it determine whether more actions are needed, or whether it is okay to finish.

The brains of a LangChain agent are an LLM. It is the LLM that is used to reason about the best way to carry out the ask requested by a user.

In order to carry out its task, and operate on things and retrieve information, the agent has what are called Tool’s in LangChain, at its disposal. It is through these tools that it is able to interact with its environment.

The tools are basically just methods/classes the agent has access to that can do things like interact with a Stock Market index over an API, update a Google Calendar event, or run a query against a database. We can build our tools as needed, depending on the nature of tasks we are trying to carry out with the agent to fulfill.

So, at a basic level, an agent needs:

an LLM to act as its **brain**, and to give it its reasoning abilities
tools so that it can interact with the environment around it and achieve its goals

Agents are a way to **run an LLM in a loop** in order to complete a task.

Agents are defined with the following:

**Agent Type** - This defines how the Agent acts and reacts to certain events and inputs. Here, we will focus on the ReAct Agent Type.

**LLM** - The AI that actually runs your prompts.

**Tools** - These are Python functions that your Agent can call to interact with the outside world. These can be as simple or as complex as you want them to be!
**Many tools make a Toolkit**. There are many toolkits already available built-in to LangChain, here, we’ll make our own.

Agents use a combination of an LLM (or an LLM Chain) as well as a Toolkit in order to perform a predefined series of steps to accomplish a goal. For this example, we’ll create a couple of custom tools as well as LangChain’s provided DuckDuckGo search tool to create a **research agent**.


In [1]:
!pip install -q langchain langchain_community
!pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.4 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
from langchain.tools import Tool, DuckDuckGoSearchResults
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

Here's a breakdown of the imports:

requests: A popular Python library for making HTTP requests.

BeautifulSoup: A library for web scraping purposes to pull the data out of HTML and XML files.

load_dotenv: A method to load environment variables from a .env file.

LangChain specific imports: These are specific to the LangChain framework and are used to define tools, prompts, chat models, chains, and agents.

DuckDuckGoSearchResults: searches over internet and ensures privacy, does not track users or record search history

In [3]:
search_tool = DuckDuckGoSearchResults()

# test the tool
search_tool.invoke('Obama')

"snippet: The former First Lady denied any rift with Barack Obama and praised school counselors in her first public video since speculation about their relationship. Learn about their love story, family and legacy in this article., title: Michelle Obama makes emotional announcement amid speculation over ..., link: https://www.hellomagazine.com/celebrities/812295/michele-obama-makes-emotional-announcement-amid-divorce-speculation/, snippet: Learn about the life and achievements of Barack Obama, the 44th president of the United States and the first African American to hold that office. Find out his biography, awards, books, family, and more., title: Barack Obama Facts | Britannica, link: https://www.britannica.com/facts/Barack-Obama, snippet: A viral claim suggests that Trump's allies proposed a constitutional amendment that would allow presidents to have a third term, including Obama. But the amendment only applies to presidents who served nonconsecutive terms, like Trump., title: Obama

In [4]:
# Create the toolkit: Define the list of tools the agent can use. Keep decription informative for the LLM
# giving LLM more capability to search the internet if it cannot do it itself

tools = [
    Tool(
        name = 'DuckDuckGo Search',
        func = search_tool.invoke,
        description = 'Use this tool to search the web for up-to-date information.'
    )
]

Create tool for parsing web pages

In [5]:
import re

def extract_url(text: str) -> str:
    """Extracts the first valid URL from a given text."""
    match = re.search(r"https?://[^\s]+", text)
    return match.group(0) if match else None


def parse_html(content) -> str:
    soup = BeautifulSoup(content, 'html.parser') # pass content of web page to bs4 to parse
    text_content_with_links = soup.get_text()
    return text_content_with_links


# This sets a user-agent header for our web requests. Some websites might block requests that don't have a user-agent set,
# thinking they're from bots.
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0'
}

def fetch_web_page(url: str) -> str:
    url = extract_url(url)
    if not url:
        return "No valid URL found in the query."
    response = requests.get(url, headers=HEADERS)
    return parse_html(response.content)


# Here, we're creating a new tool using the Tool.from_function method. This tool will use our fetch_web_page function
# to fetch and parse web pages.
web_fetch_tool = Tool.from_function(
    func = fetch_web_page,
    name = 'WebFetcher',
    description = 'Fetches the content of a web page.' # get text_content_with_links
)

In [10]:
tools.append(web_fetch_tool)
tools

[Tool(name='DuckDuckGo Search', description='Use this tool to search the web for up-to-date information.', func=<bound method BaseTool.invoke of DuckDuckGoSearchResults(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'))>),
 Tool(name='WebFetcher', description='Fetches the content of a web page.', func=<function fetch_web_page at 0x7c1fc7976200>)]

In [ ]:
# Imports the getpass module, whilch allows secure input (hides input while typing)

import getpass
import os

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')



1. scrape a web page
2. summarize content of that web page

In [14]:
# Creating your own summarizer tool
prompt_template = 'Summarize the following content: {content}'

llm = ChatOpenAI(model_name='gpt-3.5-turbo-16k')
llm_chain = LLMChain(
    llm = llm,
    prompt = PromptTemplate.from_template(prompt_template)
)

summarize_tool = Tool.from_function(
    func = llm_chain.invoke,
    name = 'Summarizer',
    description = 'Summarizes a web page.'
)

tools.append(summarize_tool)
tools

[Tool(name='DuckDuckGo Search', description='Use this tool to search the web for up-to-date information.', func=<bound method BaseTool.invoke of DuckDuckGoSearchResults(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'))>),
 Tool(name='WebFetcher', description='Fetches the content of a web page.', func=<function fetch_web_page at 0x7c1fc7976200>),
 Tool(name='Summarizer', description='Summarizes a web page.', func=<bound method Chain.invoke of LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['content'], input_types={}, partial_variables={}, template='Summarize the following content: {content}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7c1fc5c570d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7c1fc5c8ac50>, model_name='gpt-3.5-turbo-16k', model_kwargs={}, openai_api_key='sk-proj-sgceKmJwBu1LNqB-srg8vr2XC2S5JJIAF7mAuxl

In [15]:
# Create the agent

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent='zero-shot-react-description',
    handle_parsing_errors=True,  # if output is not in proper format and can be different every time, don't throw error -- handle it
    verbose=True
)

In [16]:
# Test the agent with a query

response = agent.invoke('what are the latest advancements in AI?')
print(response)

# because "latest" is used, chatgpt knows it has to search the web



> Entering new AgentExecutor chain...
I should search the web for the latest advancements in AI.
Action: DuckDuckGo Search
Action Input: 'latest advancements in AI'
Observation: snippet: Discover the 10 major AI trends set to reshape 2025: from augmented working and real-time decision-making to advanced AI legislation and sustainable AI initiatives., title: The 10 Biggest AI Trends Of 2025 Everyone Must Be Ready For Today - Forbes, link: https://www.forbes.com/sites/bernardmarr/2024/09/24/the-10-biggest-ai-trends-of-2025-everyone-must-be-ready-for-today/, snippet: Meta has released massive data sets and models that could help scientists use AI to discover new materials much faster, and in December, Hugging Face, together with the startup Entalpic, launched ..., title: What's next for AI in 2025 - MIT Technology Review, link: https://www.technologyreview.com/2025/01/08/1109188/whats-next-for-ai-in-2025/, snippet: "AI has become synonymous with large language models, but that's just on

In [17]:
response = agent.invoke('what are the latest offers on Zomato?')
print(response)



> Entering new AgentExecutor chain...
I should search for the latest offers on Zomato.
Action: DuckDuckGo Search
Action Input: 'latest offers on Zomato'
Observation: snippet: Use Zomato New User coupon code, Credit Card Offers for Additional discounts. English Edition. ... Latest Zomato Coupons, Offers (March 2025) Zomato Offers Discount Valid Till; Grab ₹ 125 Off on Burger Singh: Flat ₹ 125 Off: 31st Mar, 2025: Get Up To 20% Off on Harlequin: UP TO 20% Off:, title: Zomato Coupons, Offers: Upto 60% OFF Today - Gadgets 360, link: https://www.gadgets360.com/deals/zomato-offers-coupons, snippet: Get the latest 2 active zomato.com coupon codes, discounts and promos. Today's top deal: Take 60% Off w/ Zomato Coupon Code. Use these discount codes and save $$$! ... Our deals database is updated daily, with the last verification for Zomato's offers done on Jul 04, 2024. Trust DealDrop for rigorously vetted coupons, promo codes, sales, and ..., title: 60% Off Zomato Promo Codes (2 Working Code